# Example: Bringing Multiple Pythons To A Problem


## Setting The Stage With A Pretend Scenario

For demoing I wanted to find an example of something non-trivial to simulate an actual coding situation where writing straight CPython is insufficient performance wise and you have to start looking for solutions with Python alternatives.
 
### Scenario 
Your favorite band, 2 Pegasus Tasks, just released a new single. However, in their usual style, they did it with a twist.

Instead of distributing the song on Spotify or some other distribution platform, for its initial release they segmented  the audio file binary into tiny substrings and then inserted them according to a     secret pattern into a bunch of longer strings to create an internet scavenger hunt. 

Any group that is able to piece the song together gets a cash prize on the condition that they don't share it or how t    hey got it with others. They have provided a preliminary list of 300 strings, 100 of which the've confirmed do contain a chunk of the song, and 200 of which don't.

### Your Mission

You and a group of friends have managed to scrape down all known instances of these string sequences and tried some linear but you can't make heads or tails of what they mean. When you analyze the seque    nces by hand, sometimes you think you've found a pattern but then another example breaks it. It definitely looks like the function for the pattern will be non-linear in nature.

So you have the idea to use a Support Vector Machine based on a string kernel to learn a boundary between what is part of the pattern and what is not in higher dimensional space. You find a whitepaper about string subsequence kernels that looks like just what you need. You search through libraries online like Scikit Learn and NLTK but in this imaginary scenario, you can't find a decent implemntation of this kernel function anywhere, so you'll have to write your own.


### Example 1: Just CPython

After pouring over the whitepaper and mayber some supplemental material to make sure you understand the concepts, you are able to create an implementation of the string-subsequence-kernel. You are ready to run it, let's simulate that by going to an older revision of our example code which is just written in CPython with some profiling lines to show us how fast it is.

In [ ]:
import numpy as np
import timeit

In [ ]:
"""
this code is derivative of https://github.com/helq/python-ssk/commit/6acee597ff37f7e7e12dd8651421a4d34c5dad70
by https://github.com/helq,  which is licensed under Creattive Commons Zero v1.0 Universal / 
changes: removing lodhi assertions because we know it works, changing script handling into function, 
adding timeit code to measure duration, changing inputs to exagerrate performance problem. 
Do check that repo out if you actually want to learn about ssk/get a fast implementation for practical purposes.)
"""
# Kernel defined by Lodhi et al. (2002)
def ssk(s, t, n, lbda, accum=False):
    dynamic = {}

    def k_prim(s, t, i):
        # print( "k_prim({},{},{})".format(s, t, i) )
        if i == 0:
            # print( "k_prim({},{},{}) => 1".format(s, t, i)  )
            return 1.
        if min(len(s), len(t)) < i:
            # print( "k_prim({},{},{}) => 0".format(s, t, i)  )
            return 0.
        if (s,t,i) in dynamic:
            return dynamic[(s,t,i)]

        x = s[-1]
        s_ = s[:-1]
        indices = [i for i, e in enumerate(t) if e == x]
        toret = lbda * k_prim(s_, t, i) \
              + sum( k_prim(s_, t[:j], i-1) * (lbda**(len(t)-j+1)) for j in indices )
        # print( "k_prim({},{},{}) => {}".format(s, t, i, toret) )
        dynamic[(s,t,i)] = toret
        return toret

    def k(s, t, n):
        # print( "k({},{},{})".format(s, t, n) )
        if n <= 0:
            raise "Error, n must be bigger than zero"
        if min(len(s), len(t)) < n:
            # print( "k({},{},{}) => 0".format(s, t, n) )
            return 0.
        x = s[-1]
        s_ = s[:-1]
        indices = [i for i, e in enumerate(t) if e == x]
        toret = k(s_, t, n) \
              + lbda**2 * sum( k_prim(s_, t[:j], n-1) for j in indices )
        # print( "k({},{},{}) => {}".format(s, t, n, toret) )
        return toret

    if accum:
        toret = sum( k(s, t, i) for i in range(1, min(n,len(s),len(t))+1) )
    else:
        toret = k(s, t, n)

    # print( len(dynamic) )
    return toret

def string_kernel(xs, ys, n, lbda):
    if len(xs.shape) != 2 or len(ys.shape) != 2 or xs.shape[1] != 1 or ys.shape[1] != 1:
        raise "The shape of the features is wrong, it must be (n,1)"

    lenxs, lenys = xs.shape[0], ys.shape[0]

    mat = np.zeros( (lenxs, lenys) )
    for i in range(lenxs):
        for j in range(lenys):
            mat[i,j] = ssk(xs[i,0], ys[j,0], n, lbda, accum=True)

    mat_xs = np.zeros( (lenxs, 1) )
    mat_ys = np.zeros( (lenys, 1) )

    for i in range(lenxs):
        mat_xs[i] = ssk(xs[i,0], xs[i,0], n, lbda, accum=True)
    for j in range(lenys):
        mat_ys[j] = ssk(ys[j,0], ys[j,0], n, lbda, accum=True)

    return np.divide(mat, np.sqrt(mat_ys.T * mat_xs))

def evaluate_ssk():
    print("Testing...")
    
    ## code for the pretend scenario, long binary sequences
    s1 = np.array(["101110010010111110101011100000101000010010111100101011010011011010110111", \
                   "101000010010111111111111110000010100001001011110010101101001101101011011", \
                   "101000010010111110101011100011111111101001011110010101101001101101011011", \
                   "10111111001011111010101110000010100001001011110010101101001101101011011"]).reshape((4,1))
    s2 = np.array(["10100001001011111111111110000010100001001011110010101101001101101011011", \
                   "10100001001011111010101110000010100001001011110010111111111101101011011", \
                   "10100001001011111010101110000010100011101011110010101101001101101011011", \
                   "10100001001011111010101110110010100001001011110010101101001111111011011"]).reshape((4, 1))

    # code for pretend scenario, we are looking for common substrings up to 8 chars in length, because that could be a byte
    print( string_kernel(s1, s2, 11, 1.) )
    
print(f"Running String Kernel on the strings took: \
{timeit.timeit('evaluate_ssk()', setup='from __main__ import evaluate_ssk', number=1)} seconds", )

### CPython Is Too Slow. . . 

Oh, no! Don't know what it shows on your machine but mine had ~16 seconds and that is no bueno. The real strings were much longer than that, and the SVM is goingo to need to pairwise compare thousands of them at maybe even longer subsequence values. If you want to be able to iterate and tune your SVM model quickly, we need to build a better mousetrap.

What have we got? Well, Pypy is supposed to be a faster drop in replacement for CPython right? And even though we know it won't help us with the already optimized numpy operations, looks like a lot of the bottleneck is good old fashioned python for loops. Pypy should be able to figure out how to compile those down to machine code for us.

## Pypy To The Rescue?

So For This Example, we are going to run the same code, but swap out our Python 3 kernel in jupyter for a Pypy one. here are the steps I followed to achieve this, which should hopefully work for you if you are following along with the conda-based setup I provided in the README. they are based on this answer in Stack Overflow by jadelord on May 24, 2019: https://stackoverflow.com/questions/33850577/is-it-possible-to-run-a-pypy-kernel-in-the-jupyter-notebook

from your terminal: 
`conda install -c anaconda ipykernel`
`ipython kernel install --user --name=PyPy3`

verify that it shows up with: 
`jupyter kernelspec list`

Then you can run the above cell with Pypy by going to the jupyter menu, Kernel -> Change Kernel -> Pypy3

### How did it go? 

For me, it was around the same amount of time as CPython, if not a bit slower. Hard to know why for sure just by looking at it if we aren't familiar with the exact things Pypy is better or worse at. 

Maybe it is because this isn't actually that much code, so the JIT compiler's warm up time eats up all the optimizations we get from it? Maybe because Pypy3 is actually slower to run some of the numpy functions? Maybe there is code in here that CPython's compiler optimizes down which the Pypy interpreter is missing. All interesting thoughts. Maybe PyPy3 would be faster if we were doing this more times or with longer sequences? Let's just keep it in our back pocket for now.


## OK, what about Cython? (Spoiler alert, helq's original solution so it probably works well)

Ok, so at this point in our pretend scenario, our first optimization attempt hasn't worked out but we haven't lost hope. If these loops were running in C code that had been dynamically linked instead of going through the process of being bytecode processed through a loop, they'd probably be significantly faster. But we do need to rewrite the code a bit to get that optimization benefit. At least it is still python though.

Also a note, there are other clever things done in this below code that you wouldn't necessarily have to do to get the speed up that comes with Cython, if you plan to peruse it yourself and determine how it is all working. Helq mentions they got a trick from the Shogun C++ implementation of string ssk as well.

First, let the jupyter notebook know you are going to use you local installation of Cython for compilation with a magic string:

In [ ]:
%load_ext Cython

and here is the new code complete with some cython annotations and cdef statements for the python types. Also notice another magic string that says we would like to compile this cell with  cython

In [ ]:
%%cython

"""
This cell and the one below it are derivative of https://github.com/helq/python-ssk
by https://github.com/helq,  which is licensed under Creattive Commons Zero v1.0 Universal / 
changes: consolidating the pyx and main.py modules into this notebook, moved module scoped code into functions and added
timer code to measure duration.
"""

import numpy as np
cimport numpy as np

from cpython cimport array
import array

cimport cython

def ssk(s, t, int n, float lbda, accum=False):
    """s and t are strings, either numpy.str_ or python str, or a list of chars"""
    s_array = array.array('l', [ord(c) for c in s])
    t_array = array.array('l', [ord(c) for c in t])
    return ssk_array(s_array, t_array, n, lbda, accum)

# Kernel defined by Lodhi et al. (2002)
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
def ssk_array(array.array s_, array.array t_, int n, float lbda, accum=False):
    cdef int lens, lent
    cdef int i, sj, tk
    cdef float toret
    cdef long[:] s # this reduces the overhead 10x fold!!!
    cdef long[:] t

    s = s_ if s_.typecode == 'l' else array.array('l', [int(c) for c in s_])
    t = t_ if t_.typecode == 'l' else array.array('l', [int(c) for c in t_])

    lens, lent = len(s), len(t)
    #k_prim = (-1)*np.ones( (n+1, lens, lent) )
    cdef np.ndarray[np.float64_t, ndim=3] \
        k_prim = np.zeros((n, lens, lent), dtype=np.float64)

    k_prim[0,:,:] = 1

    for i in range(1,n):
        for sj in range(i,lens):
            toret = 0.
            for tk in range(i,lent):
                if s[sj-1]==t[tk-1]: # trick taken from shogun implemantion of SSK
                    toret = lbda * (toret + lbda*k_prim[i-1,sj-1,tk-1])
                else:
                    toret *= lbda
                k_prim[i,sj,tk] = toret + lbda * k_prim[i, sj-1, tk]

    cdef int start = 0 if accum else n-1
    cdef float k = 0.

    for i in range(n):
        for sj in range(i,lens):
            for tk in range(i,lent):
                if s[sj]==t[tk]:
                    k += lbda*lbda*k_prim[i,sj,tk]

    # print( [len(list(i for (sj,tk,i) in k_prim if i==m-1)) for m in range(n)] )
    return k

def string_kernel(xs, ys, n, lbda):
    """xs and ys are numpy arrays of strings or arrays of ints, n an integer and lbda a bool"""
    if len(xs.shape) != 2 or len(ys.shape) != 2 or xs.shape[1] != 1 or ys.shape[1] != 1:
        raise "The shape of the features is wrong, it must be (n,1)"

    cdef int lenxs, lenys
    cdef int i, j
    cdef np.ndarray[np.float64_t, ndim=2] mat, mat_xs, mat_ys
    lenxs, lenys = xs.shape[0], ys.shape[0]

    mat = np.zeros((lenxs, lenys))

    ssk_fun = ssk_array if xs.dtype == 'O' and isinstance(xs[0,0], array.array) else ssk

    # If both lists are equal, then the resulting matrix is symetric, there is no need to
    # calculate the hole thing
    if lenxs == lenys and np.array_equal(xs, ys):
        for i in range(lenxs):
            for j in range(i,lenys):
                mat[j,i] = mat[i,j] = ssk_fun(xs[i,0], ys[j,0], n, lbda, accum=True)

        mat_xs = mat_ys = mat.diagonal().reshape((lenxs, 1))

    else:
        for i in range(lenxs):
            for j in range(lenys):
                mat[i,j] = ssk_fun(xs[i,0], ys[j,0], n, lbda, accum=True)

        mat_xs = np.zeros((lenxs, 1))
        mat_ys = np.zeros((lenys, 1))

        for i in range(lenxs):
            mat_xs[i] = ssk_fun(xs[i,0], xs[i,0], n, lbda, accum=True)
        for j in range(lenys):
            mat_ys[j] = ssk_fun(ys[j,0], ys[j,0], n, lbda, accum=True)

    return np.divide(mat, np.sqrt(mat_ys.T * mat_xs))

In [ ]:
def evaluate_cython_ssk():    
    print("Testing...")
    
     ## code for the pretend scenario, long binary sequences
    s1 = np.array(["101110010010111110101011100000101000010010111100101011010011011010110111", \
                   "101000010010111111111111110000010100001001011110010101101001101101011011", \
                   "101000010010111110101011100011111111101001011110010101101001101101011011", \
                   "10111111001011111010101110000010100001001011110010101101001101101011011"]).reshape((4,1))
    s2 = np.array(["10100001001011111111111110000010100001001011110010101101001101101011011", \
                   "10100001001011111010101110000010100001001011110010111111111101101011011", \
                   "10100001001011111010101110000010100011101011110010101101001101101011011", \
                   "10100001001011111010101110110010100001001011110010101101001111111011011"]).reshape((4, 1))

    # code for pretend scenario, we are looking for common substrings up to 8 chars in length, because that could be a byte
    print( string_kernel(s1, s2, 11, 1.) )
    
print(f"Running String Kernel on the strings took: \
{timeit.timeit('evaluate_cython_ssk()', setup='from __main__ import evaluate_cython_ssk', number=1)} seconds", )


Ah, that's more like it. Much faster. Wow!

#### Note, as long as you recompile the Cython cell, you can run this with either CPython3 or Pypy3

That should be fast enough, now imaginary you can take this kernel function and build your Support Vector Machine and find the missing parts of your song!

Thanks for trying this out! Hope it gave a sense for trying out different python variants in search of a solution.